## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key
from config import weather_api_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Forecast
0,0,Mahebourg,MU,2021-04-04 22:38:20,-20.4081,57.7000,80.01,77,35,8.01,light rain
1,1,Atuona,PF,2021-04-04 22:38:20,-9.8000,-139.0333,80.56,79,75,8.84,broken clouds
2,2,Saint-Pierre,RE,2021-04-04 22:33:29,-21.3393,55.4781,77.00,78,0,5.75,clear sky
3,3,Campbellsville,US,2021-04-04 22:38:21,37.3434,-85.3419,69.80,30,1,6.91,clear sky
4,4,Kudahuvadhoo,MV,2021-04-04 22:38:21,2.6708,72.8944,83.28,75,7,14.16,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Forecast
0,0,Mahebourg,MU,2021-04-04 22:38:20,-20.4081,57.7000,80.01,77,35,8.01,light rain
1,1,Atuona,PF,2021-04-04 22:38:20,-9.8000,-139.0333,80.56,79,75,8.84,broken clouds
2,2,Saint-Pierre,RE,2021-04-04 22:33:29,-21.3393,55.4781,77.00,78,0,5.75,clear sky
4,4,Kudahuvadhoo,MV,2021-04-04 22:38:21,2.6708,72.8944,83.28,75,7,14.16,clear sky
13,13,Vaini,TO,2021-04-04 22:38:24,-21.2000,-175.2000,78.80,83,20,14.97,few clouds
17,17,Rikitea,PF,2021-04-04 22:38:25,-23.1203,-134.9692,77.79,66,51,18.30,broken clouds
26,26,Luanda,AO,2021-04-04 22:37:07,-8.8368,13.2343,78.80,83,20,5.75,few clouds
27,27,Alofi,NU,2021-04-04 22:38:29,-19.0595,-169.9187,82.40,78,100,10.36,moderate rain
28,28,Hadejia,NG,2021-04-04 22:38:29,12.4498,10.0444,80.38,12,4,10.25,clear sky
44,44,Sur,OM,2021-04-04 22:38:34,22.5667,59.5289,83.32,43,2,7.65,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID       203
City          203
Country       203
Date          203
Lat           203
Lng           203
Max Temp      203
Humidity      203
Cloudiness    203
Wind Speed    203
Forecast      203
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID       203
City          203
Country       203
Date          203
Lat           203
Lng           203
Max Temp      203
Humidity      203
Cloudiness    203
Wind Speed    203
Forecast      203
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Forecast", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Forecast,Lat,Lng,Hotel Name
0,Mahebourg,MU,80.01,light rain,-20.4081,57.7000,
1,Atuona,PF,80.56,broken clouds,-9.8000,-139.0333,
2,Saint-Pierre,RE,77.00,clear sky,-21.3393,55.4781,
4,Kudahuvadhoo,MV,83.28,clear sky,2.6708,72.8944,
13,Vaini,TO,78.80,few clouds,-21.2000,-175.2000,
17,Rikitea,PF,77.79,broken clouds,-23.1203,-134.9692,
26,Luanda,AO,78.80,few clouds,-8.8368,13.2343,
27,Alofi,NU,82.40,moderate rain,-19.0595,-169.9187,
28,Hadejia,NG,80.38,clear sky,12.4498,10.0444,
44,Sur,OM,83.32,clear sky,22.5667,59.5289,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
   # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

hotels

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


{'html_attributions': [],
 'next_page_token': 'ATtYBwJbHfQkl6ShzX2Llp_yWvRhT6mr-Zo6WyegN5bnjGWqGfDJXuqtSE3sNYJjNz_UK0HT5gD5aeFV-vT3w1nYM90swQ69eCoE4BOKkTKB0BBbvFXmeDafbAviGEZuW92zrbI7q5d2h7ofTr1E1HMJLCfW-98X9Ei4kJGJjkBRpHcmLu1rhxMG398Z0ZyX4_SlOV2FFPp9Is0VW1nbW341SBEuUoOwTs0u91UjssybDV7V-qsbEb1gtLsq0vGLmty4Xq9WkbcTrDZeIyxv8y_l4G9PkcNd_2D7r7W2dagcq7ZdSkqacUgqKTryuIxH-PGv_53w5Cp9KWnN6xF6Mfgk0S0RW2CMGK7YAXclCmeby_FYJyTyMVwfMq8GAfM475ctP-aK3eOkJr9cvCPqFF5ue_fgpbV3vGsQ3e18LNGGhRGlveEmTbbssvu6',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 15.74286240000001, 'lng': -96.4658254},
    'viewport': {'northeast': {'lat': 15.74422898029151,
      'lng': -96.46450586970849},
     'southwest': {'lat': 15.7415310197085, 'lng': -96.4672038302915}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'name': 'Hotel Posada San Jose',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 2962,
     'html_attribution

In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Forecast,Lat,Lng,Hotel Name
0,Mahebourg,MU,80.01,light rain,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
1,Atuona,PF,80.56,broken clouds,-9.8000,-139.0333,Villa Enata
2,Saint-Pierre,RE,77.00,clear sky,-21.3393,55.4781,Lindsey Hôtel
4,Kudahuvadhoo,MV,83.28,clear sky,2.6708,72.8944,Niyama Private Islands Maldives
13,Vaini,TO,78.80,few clouds,-21.2000,-175.2000,Keleti Beach Resort
17,Rikitea,PF,77.79,broken clouds,-23.1203,-134.9692,Pension Maro'i
26,Luanda,AO,78.80,few clouds,-8.8368,13.2343,LANDU MARINEZA LUFUANDU TONI
27,Alofi,NU,82.40,moderate rain,-19.0595,-169.9187,Taloa Heights
28,Hadejia,NG,80.38,clear sky,12.4498,10.0444,Kasuwar Kuda Hadejia General Hospital Road
44,Sur,OM,83.32,clear sky,22.5667,59.5289,Sur Plaza Hotel


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))